In [19]:
import numpy as np
from torch import Tensor
from sklearn.model_selection import KFold
from MLCode.utils import load_cup_data, np_cup_TR, plot_NN_TR_VAL
from sklearn.preprocessing import StandardScaler
from MLCode.NN import NN_HyperParameters
from MLCode.NN_cup import NN_Regressor, train_NN_cup, save_training


import matplotlib.pyplot as plt

df = load_cup_data()
X, Y = np_cup_TR(df)
Y_scaler = StandardScaler()
Y_scaler.fit(Y)
X, Y = Tensor(X), Tensor(Y)

# 10% test set
test_samples = X.shape[0] // 10

X_dev = X[:-test_samples]
Y_dev = Y[:-test_samples]

X_test = X[-test_samples:]
Y_test = Y[-test_samples:]

# 20% validation
val_samples = X_dev.shape[0] // 5
X_train = X_dev[:-val_samples]
Y_train = Y_dev[:-val_samples]

X_val = X_dev[-val_samples:]
Y_val = Y_dev[-val_samples:]

# train with K-fold cross-validation
def train_NN_K_validation(model, k_folds:int, X_dev, Y_dev, iters:int=None):
    """Train NN with k-fold cross-validation. If `iters` is set,
    instead of using all the folds, it will reach a number of `iters`
    training iterations.

    Returns `(MEE_mean, MEE_var, seconds, converged:Bool)`
    """
    kf = KFold(k_folds)
    val_errors = []
    seconds = 0
    converged = True
    for train_index, val_index in kf.split(X_dev):
        X_train, Y_train = X_dev[train_index], Y_dev[train_index]
        X_val, Y_val = X_dev[val_index], Y_dev[val_index]
        stats = train_NN_cup(net, X_train, Y_train, X_val, Y_val, 15,200,False)
        MEE, sec, conv = stats
        val_errors.append(MEE)
        seconds += sec
        converged = converged and conv
        if iters:
            iters -= 1
            if iters == 0: break

    val_errors = np.array(val_errors)
    MEE_mean = np.mean(val_errors)
    MEE_var = np.var(val_errors)
    return MEE_mean, MEE_var, seconds, converged

from pathlib import Path
from MLCode.conf import path_data
import pandas as pd

def get_model_val_err(directory: Path):
    """Returns validation error (mean and variance) from saved model directory.
    """
    info_file = directory / Path('infos.txt')
    info_text = info_file.read_text()
    lines = info_text.split('\n')
    mean_line = [x for x in lines if x.startswith('Val error') ][0]
    var_line = [x for x in lines if x.startswith(' measured') ][0]
    n_char_mean = len('Val error achieved: ')
    n_char_var = len('measured with variance: ')
    return float(mean_line[n_char_mean:]), float(var_line[n_char_var:])

def saved_NN_models():
    """Returns a pandas DataFrame with `model_name, MEE_mean, MEE_var`
    for each saved NN model."""

    NN_directory = path_data / Path('NN_training')
    models = []
    for dir in NN_directory.iterdir():
        val_err = get_model_val_err(dir)
        model = (dir.name, val_err[0], val_err[1])
        models.append(model)
    
    return pd.DataFrame(models, columns=['model_name', 'MEE_mean', 'MEE_var'])

In [41]:
NN_HP = NN_HyperParameters(
    [10, 200, 200, 200],
    lr=0.0015,
    beta1=0.9,
    beta2=0.999,
    weight_decay=0,
    mb_size=32,
)
net = NN_Regressor(2,NN_HP, Y_scaler)


# stats = train_NN_cup(net, X_train, Y_train, X_val, Y_val, 15,100,False)
stats = train_NN_K_validation(net, 5, X_dev, Y_dev)
save_training(stats, NN_HP)

layers = [ 10, 100, 100, 100,]
lr = 0.0015
beta1 = 0.9
beta2 = 0.999
weight_decay = 0
mb_size = 32

Val error achieved: 1.9611931337838784
 measured with variance: 0.5149920914470603
Time (seconds): 63.38024250499984
Convergence: True


In [42]:
df = saved_NN_models()
df = df.sort_values(by=['MEE_mean'])
df




,model_name,MEE_mean,MEE_var
17,2x1200_1.0E-03,1.747238,0.584657
21,2x1200_5.0E-04,1.821535,0.650594
6,2x1000_1.0E-03,1.840667,0.700100
10,2x800_5.0E-04,1.850379,0.537710
19,2x1000_5.0E-04,1.877395,0.665915
8,2x1500_5.0E-04,1.883832,0.652425
18,2x800_1.5E-03,1.901263,0.460280
11,2x800_1.0E-03,1.925140,0.617907
7,2x2000_1.0E-03,1.931692,0.516322
20,2x2000_1.5E-03,1.950580,0.465322


In [ ]:
array = df['MEE_mean'].to_numpy()
array = array.reshape(-1,4).transpose()
pd.DataFrame(array)

In [78]:
array = df['model_name'].to_numpy()
array = array.reshape(-1,4).transpose()
pd.DataFrame(array)

,0,1,2,3,4,5,6
0,1000_1.0E-01,1000x1000_1.0E-01,100_1.0E-01,100x100_1.0E-01,1500x1500_1.0E-01,500_1.0E-01,500x500_1.0E-01
1,1000_1.0E-02,1000x1000_1.0E-02,100_1.0E-02,100x100_1.0E-02,1500x1500_1.0E-02,500_1.0E-02,500x500_1.0E-02
2,1000_1.0E-03,1000x1000_1.0E-03,100_1.0E-03,100x100_1.0E-03,1500x1500_1.0E-03,500_1.0E-03,500x500_1.0E-03
3,1000_1.0E-04,1000x1000_1.0E-04,100_1.0E-04,100x100_1.0E-04,1500x1500_1.0E-04,500_1.0E-04,500x500_1.0E-04
